In [1]:
import requests

def get_wind_forecast(lat, lon):
    """
    Fetch wind forecast data from NOAA's NWS API.

    Parameters:
        lat (float): Latitude of the location.
        lon (float): Longitude of the location.

    Returns:
        dict: Wind forecast data.
    """
    # Base URL for NWS Points Forecast API
    base_url = "https://api.weather.gov/points"
    
    try:
        # Step 1: Get the forecast URL for the given coordinates
        response = requests.get(f"{base_url}/{lat},{lon}")
        response.raise_for_status()  # Raise exception for HTTP errors
        data = response.json()

        forecast_url = data["properties"]["forecastHourly"]
        
        # Step 2: Fetch hourly forecast data
        forecast_response = requests.get(forecast_url)
        forecast_response.raise_for_status()
        forecast_data = forecast_response.json()

        # Step 3: Extract wind forecast information
        wind_forecast = []
        for period in forecast_data["properties"]["periods"]:
            wind_forecast.append({
                "time": period["startTime"],
                "wind_speed": period["windSpeed"],
                "wind_direction": period["windDirection"]
            })

        return wind_forecast
    
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data: {e}")
        return None



In [2]:

# Replace with your desired coordinates
latitude = 33.971727  # Example: Los Angeles
longitude = -118.4556527

wind_data = get_wind_forecast(latitude, longitude)
if wind_data:
    print("Wind Forecast:")
    for entry in wind_data:
        print(f"Time: {entry['time']}, Speed: {entry['wind_speed']}, Direction: {entry['wind_direction']}")


Wind Forecast:
Time: 2024-12-03T10:00:00-08:00, Speed: 5 mph, Direction: WSW
Time: 2024-12-03T11:00:00-08:00, Speed: 5 mph, Direction: WSW
Time: 2024-12-03T12:00:00-08:00, Speed: 5 mph, Direction: WSW
Time: 2024-12-03T13:00:00-08:00, Speed: 5 mph, Direction: WSW
Time: 2024-12-03T14:00:00-08:00, Speed: 5 mph, Direction: WSW
Time: 2024-12-03T15:00:00-08:00, Speed: 5 mph, Direction: WSW
Time: 2024-12-03T16:00:00-08:00, Speed: 10 mph, Direction: W
Time: 2024-12-03T17:00:00-08:00, Speed: 10 mph, Direction: W
Time: 2024-12-03T18:00:00-08:00, Speed: 10 mph, Direction: W
Time: 2024-12-03T19:00:00-08:00, Speed: 10 mph, Direction: W
Time: 2024-12-03T20:00:00-08:00, Speed: 10 mph, Direction: W
Time: 2024-12-03T21:00:00-08:00, Speed: 10 mph, Direction: W
Time: 2024-12-03T22:00:00-08:00, Speed: 0 mph, Direction: 
Time: 2024-12-03T23:00:00-08:00, Speed: 0 mph, Direction: 
Time: 2024-12-04T00:00:00-08:00, Speed: 0 mph, Direction: 
Time: 2024-12-04T01:00:00-08:00, Speed: 0 mph, Direction: 
Time: 2024-

In [15]:
import requests
import datetime

def download_grib_file(min_lat, max_lat, min_lon, max_lon, forecast_hour, output_file):
    """
    Downloads a GRIB2 file from NOAA for a specified area and forecast hour.

    Parameters:
        min_lat (float): Minimum latitude of the area.
        max_lat (float): Maximum latitude of the area.
        min_lon (float): Minimum longitude of the area.
        max_lon (float): Maximum longitude of the area.
        forecast_hour (int): Forecast hour (e.g., 240 for 10-day forecast).
        output_file (str): Path to save the downloaded GRIB2 file.
    """
    # Get the current UTC time
    now = datetime.datetime.now(datetime.timezone.utc)
    run_date = now.strftime("%Y%m%d")
    run_hour = "00"  # Default to 00 UTC; change as needed

    # Construct the URL for the GRIB2 file
    file_name = f"gfs.t{run_hour}z.pgrb2b.0p25.f{forecast_hour:03d}"
    base_url = f"https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/"
    file_url = f"{base_url}gfs.{run_date}/{run_hour}/atmos/{file_name}"

    try:
        # Request to download the GRIB2 file
        response = requests.get(file_url, stream=True)
        response.raise_for_status()  # Check for HTTP errors

        # Save the file locally
        with open(output_file, "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                file.write(chunk)

        print(f"GRIB2 file downloaded successfully: {output_file}")
    except requests.exceptions.RequestException as e:
        print(f"Error downloading GRIB2 file: {e}")
        print(f"URL attempted: {file_url}")


In [61]:
import datetime

def get_latest_grib_time():
    """
    Calculates the latest GRIB file base time in the format required for downloading.

    Returns:
        str: The latest base time in 'YYYYMMDD/HH' format.
    """
    # Get the current UTC time
    now = datetime.datetime.now(datetime.UTC)
    print(f"Current UTC time: {now.strftime('%c')}")

    # GFS model updates every 6 hours (00, 06, 12, 18 UTC)
    # Find the most recent model run hour
    recent_run_hour = (now.hour // 6) * 6
    recent_run_time = now.replace(hour=recent_run_hour, minute=0, second=0, microsecond=0)

    # If the current time is before the most recent run's availability, go back one cycle
    # GFS files are typically available ~4 hours after the run starts
    if now < recent_run_time + datetime.timedelta(hours=4):
        recent_run_time -= datetime.timedelta(hours=6)

    # Format as 'YYYYMMDD/HH'
    return recent_run_time.strftime("%Y%m%d"), recent_run_time.strftime("%H")

# Example usage
latest_grib_time = get_latest_grib_time()
print(f"Latest GRIB time: {latest_grib_time}")

Current UTC time: Wed Dec  4 00:24:01 2024
Latest GRIB time: ('20241203', '18')


In [59]:
now

datetime.datetime(2024, 12, 4, 0, 7, 11, 899654, tzinfo=datetime.timezone.utc)

In [58]:
now.strftime("%H")

'00'

In [60]:

# Example usage
# Define the geographic bounds (Note: bounds are ignored for GRIB2b files)
min_lat = 32.0
max_lat = 34.5
min_lon = -122.0
max_lon = -116.0

# Specify the forecast hour (e.g., 240 for 10 days) and output file
forecast_hour = 240
output_path = "gfs_10day_forecast.grib2"

# Download the GRIB2 file
download_grib_file(min_lat, max_lat, min_lon, max_lon, forecast_hour, output_path)


Error downloading GRIB2 file: 404 Client Error: Not Found for url: https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs.20241204/00/atmos/gfs.t00z.pgrb2b.0p25.f240
URL attempted: https://nomads.ncep.noaa.gov/pub/data/nccf/com/gfs/prod/gfs.20241204/00/atmos/gfs.t00z.pgrb2b.0p25.f240


In [35]:
min_lon = -122.0 + 360
max_lon = -116.0 + 360
print(min_lon,max_lon)

238.0 244.0


In [ ]:
# docs here: https://nomads.ncep.noaa.gov/gribfilter.php?ds=gfs_0p25_1hr

def download_nomads_gfs_forecast_file():
    date = 
url = https://nomads.ncep.noaa.gov/cgi-bin/filter_gfs_0p25_1hr.pl?dir=%2Fgfs.20241203%2F12%2Fatmos&file=gfs.t12z.pgrb2.0p25.f240&var_UGRD=on&var_VGRD=on&lev_10_m_above_ground=on&subregion=&toplat=34.5&leftlon=238&rightlon=244&bottomlat=32

In [29]:
import pygrib
import numpy as np
from math import atan2, degrees, sqrt

def get_wind_at_location(grib_file, lat, lon, forecast_time):
    """
    Extracts wind speed and direction at a given GPS coordinate and time from a GRIB2 file.

    Parameters:
        grib_file (str): Path to the GRIB2 file.
        lat (float): Latitude of the location.
        lon (float): Longitude of the location.
        forecast_time (int): Forecast hour to extract data for.

    Returns:
        dict: A dictionary with wind speed (m/s) and direction (degrees).
    """
    try:
        # Open the GRIB2 file
        with pygrib.open(grib_file) as grbs:
            # Filter messages for U and V wind components at 10m above ground and the given forecast time
            u_msg = grbs.select(name="U component of wind", level=100, forecastTime=forecast_time)[0]
            v_msg = grbs.select(name="V component of wind", level=1000, forecastTime=forecast_time)[0]

            # Extract the data and corresponding lat/lon grid
            u_data, lats, lons = u_msg.data(lat1=lat, lat2=lat, lon1=lon, lon2=lon)
            v_data, _, _ = v_msg.data(lat1=lat, lat2=lat, lon1=lon, lon2=lon)

            # Interpolate U and V components at the exact point
            u_value = u_data[0][0]
            v_value = v_data[0][0]

            # Calculate wind speed (m/s) and direction (degrees)
            wind_speed = sqrt(u_value**2 + v_value**2)
            wind_direction = (270 - degrees(atan2(v_value, u_value))) % 360

            return {"wind_speed": wind_speed, "wind_direction": wind_direction}

    except Exception as e:
        print(f"Error processing GRIB2 file: {e}")
        return None




In [54]:
gps_lat = 33.0  # Example latitude
gps_lon = -118.0 + 360  # Example longitude
get_wind_at_location("gfs.t12z.pgrb2.0p25.f240", gps_lat, gps_lon, 240)

Error processing GRIB2 file: no matches found


In [47]:
gps_lat = 33.0  # Example latitude
gps_lon = -118.0 + 360  # Example longitude
grbs = pygrib.open("gfs.t12z.pgrb2.0p25.f240")
for grb in grbs:
    print(grb)  # Prints metadata about each message in the file
with pygrib.open("gfs.t12z.pgrb2.0p25.f240") as grbs:
    for grb in grbs:
        print(f"Forecast time: {grb.forecastTime} hours, Parameter: {grb.name}")
    u_msg = grbs.select(name="10 metre U wind component", level=10, forecastTime=240)[0]
    print(u_msg)
    print(u_msg.data())
    x = u_msg.data()

1:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 240 hrs:from 202412031200
2:10 metre V wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 240 hrs:from 202412031200
Forecast time: 240 hours, Parameter: 10 metre U wind component
Forecast time: 240 hours, Parameter: 10 metre V wind component
1:10 metre U wind component:m s**-1 (instant):regular_ll:heightAboveGround:level 10 m:fcst time 240 hrs:from 202412031200
(array([[ 0.68010986,  0.80010986,  1.22010986,  1.42010986,  1.32010986,
         1.32010986,  0.94010986,  0.33010986,  0.16010986,  0.37010986,
         0.55010986,  0.63010986,  0.14010986, -0.62989014, -1.30989014,
        -1.43989014, -1.28989014, -1.00989014, -0.61989014, -0.78989014,
        -1.65989014, -2.82989014, -3.13989014, -2.82989014, -3.00989014],
       [ 1.26010986,  1.45010986,  1.92010986,  2.09010986,  1.97010986,
         2.07010986,  1.87010986,  1.17010986,  0.56010986,  0.44010

In [53]:
x[0].shape

(11, 25)

In [52]:
x[1].shape

(11, 25)

In [37]:
grbs.select(name="U component of wind", level=10, forecastTime=0)

ValueError: no matches found

In [30]:
# Example usage
# Path to the GRIB2 file
grib_file_path = "gfs_10day_forecast.grib2"

# Specify the GPS coordinates and forecast time
gps_lat = 33.0  # Example latitude
gps_lon = -120.0  # Example longitude
forecast_hour = 0  

# Get wind speed and direction
result = get_wind_at_location(grib_file_path, gps_lat, gps_lon, forecast_hour)
if result:
    print(f"Wind speed: {result['wind_speed']} m/s")
    print(f"Wind direction: {result['wind_direction']} degrees")

Error processing GRIB2 file: no matches found


In [28]:
grib_file_path = "gfs_10day_forecast.grib2"
with pygrib.open(grib_file_path) as grbs:
    for grb in grbs:
        print(grb)  # Prints metadata about each message in the file

1:Geopotential height:gpm (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 240 hrs:from 202412030000
2:Temperature:K (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 240 hrs:from 202412030000
3:Relative humidity:% (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 240 hrs:from 202412030000
4:U component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 240 hrs:from 202412030000
5:V component of wind:m s**-1 (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 240 hrs:from 202412030000
6:Absolute vorticity:s**-1 (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 240 hrs:from 202412030000
7:Ozone mixing ratio:kg kg**-1 (instant):regular_ll:isobaricInhPa:level 100 Pa:fcst time 240 hrs:from 202412030000
8:Geopotential height:gpm (instant):regular_ll:isobaricInhPa:level 200 Pa:fcst time 240 hrs:from 202412030000
9:Temperature:K (instant):regular_ll:isobaricInhPa:level 200 Pa:fcst time 240 hrs:from 202412030000
10:Relative humidity